# Sorteo Trece Lotes

En esta libreta se explica el procedimiento del sorteo de trece lotes.

## Librerías necesarias

In [1]:
import random # Para las selecciones aleatorias
import pandas as pd # Para el manejo de los datos


# Para poder apreciar la lista completa de boletos de los participantes
pd.set_option('display.max_colwidth', None)
# Sólo para desactivar una opción de impresión vertical
%pprint

Pretty printing has been turned OFF


## Función para sortear los boletos a los participantes

In [2]:
def sorteoBoletos(participantes, semilla):
    """
        Esta función realiza el sorteo aleatorio de boletos entre todos
        los participantes. El número de boletos es igual a la sumatoria de las antigüedades de los
        participantes.

        Parámetros
        ----------
        - participantes: Objeto pd.DataFrame que contien las columnas ['num_empleado', 'antiguedad'].
        
        - semilla: Objeto <str> que se utilizará como semilla para los procesos aleatorios
                   proporcionados por la librería 'random'.
        
        Regresa
        -------
        - participantes: Objeto pd.DataFrame que contiene la asignación de los boletos para cada
                         participante. Cuenta con las columnas  
                         ['num_empleado', 'antiguedad', 'boletos].
        
        -boletos: Objeto <list> que contiene la lista de boletos desordenados que se han sorteado.
    """

    # Se establece la semilla
    random.seed(semilla)

    # ----- GENERACIÓN DE LOS BOLETOS -----

    # Obtenemos el número total de boletos
    total_boletos = participantes['antiguedad'].sum()

    # Se generan con el formato 0001, 0002, ..., 0010, 0011, ..., 0099, 0100, 0101..., 0999, 1000,...
    # Primero, se calcula el número de ceros que necesitamos de acuerdo al número
    # de dígitos que tenga el número total de boletos
    n_ceros = len(str(total_boletos))
    boletos = [str(i).zfill(n_ceros) for i in range(1, total_boletos+1)]

    # ----- MEZCLA DE LOS BOLETOS -----

    for _ in range(0, 10):
        random.shuffle(boletos)

    # ----- ASIGNACIÓN ALEATORIA DE LOS BOLETOS A CADA PARTICIPANTE -----

    # Obtenemos las antiguedades de cada participante
    antiguedades = participantes['antiguedad']

    # Lista en donde se guardará la selección de boletos para cada participante
    # y que posteriormente se añadirá al DataFrame
    boletos_df = []

    # Generamos una copia de la lista de boletos desordenada
    boletos_copia = [boleto for boleto in boletos]

    for n_antiguedad in antiguedades:
        # Se toman aleatoriamente el número de boletos correspondiente al participante actual
        seleccion_boletos = random.sample(boletos_copia, n_antiguedad)
        boletos_df.append(seleccion_boletos)
        
        # Se eliminan los boletos seleccionados
        boletos_copia = [boleto for boleto in boletos_copia if boleto not in seleccion_boletos]

    # Se añade la columna de "boletos" al dataframe de participantes
    participantes['boletos'] = boletos_df
    
    return participantes, boletos

## Función para el sorteo de los lotes

In [3]:
def sorteoTerrenos(semilla, participantes, terrenos, boletos_mezclados):
    """
        Esta función realiza el sorteo aleatorio de los terrenos.

        Parámetros
        ----------
        - semilla: Objeto <str> que se utilizará como semilla para los procesos aleatorios
                   proporcionados por la librería 'random'.

        - participantes: Objeto pd.DataFrame que contien las columnas 
                         ['num_empleado', 'antiguedad', 'boletos'].
        
        - terrenos: Objeto pd.DataFrame que contien la columna ['Numero_Lote'].

        - boletos_mezclados: Objeto <list> que contiene la lista de boletos desordenados que se han 
                             sorteado.
        
        Regresa
        -------
        - resultados: Objeto pd.DataFrame que contiene a los ganadores de los terrenos. Cuenta con
                      las columnas ['Numero_Lote', 'boleto_ganador', 'num_empleado', 
                                    'antiguedad', 'boletos']
    """

    # Se establece la semilla
    random.seed(semilla)

    # Lista para almacenar los boletos ganadores de cada terreno
    boletos_ganadores = []

    # Lista para almacenar a los participantes gandores de cada terreno
    num_empleados_ganadores = []

    # Creamos una copia de los boletos
    boletos_copia = [boleto for boleto in boletos_mezclados]

    for terreno in terrenos['Numero_Lote'].values:    
        # Se selecciona un boleto aleatoriamente y se añade a la lista de boletos ganadores
        boleto_ganador = random.choice(boletos_copia)
        boletos_ganadores.append(boleto_ganador)
        
        # Se busca el boleto ganador en la lista de boletos del DataFrame 'participantes'.
        registro_ganador = participantes[participantes['boletos'].apply(lambda x: boleto_ganador in x)]
        
        # Extraemos el número de trabajador del participante ganador y lo añadimos a la lista
        num_empleados_ganadores.append(registro_ganador['num_empleado'].values[0])
        
        # Extraemos la lista de los boletos del ganador
        boletos_a_eliminar = registro_ganador['boletos'].values[0]
        
        # Eliminamos todos los boletos del ganador de los boletos disponibles
        boletos_copia = [boleto for boleto in boletos_copia if boleto not in boletos_a_eliminar]

    # Añadimos al DataFrame de los terrenos a los ganadores
    terrenos['boleto_ganador'] = boletos_ganadores
    terrenos['num_empleado'] = num_empleados_ganadores

    # Combinamos el dataframe de los terrenos con el de los participantes
    resultados = pd.merge(terrenos, participantes, on="num_empleado", how="outer")
    resultados = resultados.head(len(terrenos.index))
    
    return resultados

## Lectura del archivo de los participantes

In [4]:
# Se lee el archivo csv de los participantes que contiene las columnas
# ['Empleado', 'Antig�edad (Antigüedad)', '']
participantes = pd.read_csv('./datos/PARTICIPANTES SORTEO 13 LOTES UNISON.csv')
participantes

,Empleado,Antig�edad,Unnamed: 2
0,97134,10,NaN
1,33154,6,NaN
2,33524,5,NaN
3,32799,7,NaN
4,97113,12,NaN
...,...,...,...
83,31617,9,NaN
84,96828,12,NaN
85,34223,3,NaN
86,29276,20,NaN


Con la finalidad de ajustar los datos proporcionados al formato en el que el software los utiliza, se realiza un renombrado de las columnas y los registros se ordenan de mayor a menor de acuerdo a la antigüedad de los participantes

In [5]:
# Se renombran las columnas
participantes.columns = ['num_empleado', 'antiguedad', '']
# Se elimina la última columna vacía, es decir, nos quedamos con las primeras dos
participantes = participantes[['num_empleado', 'antiguedad']]
# Ordenamos a los participantes de mayor a menor de acuerdo a su antiguedad
participantes = participantes.sort_values(by=['antiguedad'], ascending=[False])
# Se resetea la columna de índices (para evitar confusiones)
participantes.reset_index(drop=True, inplace=True)
participantes

,num_empleado,antiguedad
0,22722,34
1,24321,32
2,26470,28
3,27139,27
4,28526,25
...,...,...
83,32285,3
84,34203,3
85,33966,3
86,34037,3


## Lectura del archivo de los terrenos

In [6]:
terrenos = pd.read_csv('datos/terrenos.csv')
terrenos

,Numero_Lote
0,50482004
1,50481003
2,50481005
3,50480002
4,50480003
5,50480004
6,50480005
7,50480006
8,50480007
9,50479002


## Sorteo de los boletos

Esta función se utiliza en la **Fase 2: Asignación de Boletos** de la aplicación y los resultados se muestran exactamente igual que en el siguiente DataFrame

In [7]:
# Se establece una semilla
semilla = "15/12/2021 17:20"

# Se obtiene el DataFrame con los resultados del sorteo y la lista de boletos mezclados
# que posteriormente se utilizarán para sortear los terrenos.
df_sorteo_boletos, pila_boletos_mezclados = sorteoBoletos(participantes, semilla)
df_sorteo_boletos

,num_empleado,antiguedad,boletos
0,22722,34,"[706, 646, 129, 024, 927, 205, 266, 025, 769, 084, 570, 542, 071, 705, 617, 854, 560, 272, 908, 537, 281, 600, 659, 565, 588, 249, 947, 042, 886, 924, 131, 417, 564, 905]"
1,24321,32,"[716, 081, 039, 181, 326, 188, 514, 196, 292, 395, 283, 145, 072, 324, 789, 214, 657, 010, 363, 790, 642, 920, 679, 680, 014, 862, 660, 230, 316, 121, 604, 210]"
2,26470,28,"[013, 268, 586, 235, 804, 539, 757, 082, 425, 856, 236, 739, 410, 086, 133, 158, 578, 928, 096, 672, 906, 708, 844, 605, 228, 912, 918, 607]"
3,27139,27,"[127, 244, 416, 383, 191, 650, 348, 431, 036, 770, 221, 051, 029, 407, 056, 325, 260, 721, 262, 640, 301, 476, 836, 558, 101, 310, 686]"
4,28526,25,"[282, 493, 796, 441, 347, 738, 907, 601, 402, 430, 841, 702, 815, 400, 168, 904, 229, 813, 114, 468, 849, 811, 648, 867, 330]"
...,...,...,...
83,32285,3,"[017, 613, 651]"
84,34203,3,"[209, 045, 423]"
85,33966,3,"[477, 754, 929]"
86,34037,3,"[878, 574, 027]"


## Sorteo de los lotes

Esta función se utiliza en la **Fase 3: Sorteo de Lotes** de la aplicación. Antes de mostrar los resultados se eliminan las columnas que contienen las antigüedades de los participantes y la lista de sus boletos asociados. Esto se hace porque en la Fase 3 del sorteo dichas columnas ya no se necesitan.

In [8]:
# Se establece una semilla
semilla = "16/11/2021 18:08"

# Se obtiene el resultado de los lotes. El objeto resultante es la unión del 
# DataFrame que contiene el número de lote y el boleto ganador y el DataFrame que contiene
# los resultados del sorteo de los boletos.
df_sorteo_terrenos = sorteoTerrenos(semilla, df_sorteo_boletos, terrenos, pila_boletos_mezclados)
df_sorteo_terrenos

,Numero_Lote,boleto_ganador,num_empleado,antiguedad,boletos
0,50482004.0,073,31136,10,"[480, 518, 932, 447, 001, 356, 108, 073, 643, 936]"
1,50481003.0,711,32650,5,"[284, 711, 948, 795, 781]"
2,50481005.0,721,27139,27,"[127, 244, 416, 383, 191, 650, 348, 431, 036, 770, 221, 051, 029, 407, 056, 325, 260, 721, 262, 640, 301, 476, 836, 558, 101, 310, 686]"
3,50480002.0,887,29082,21,"[019, 887, 382, 897, 392, 061, 175, 837, 474, 662, 185, 208, 436, 200, 352, 684, 426, 100, 900, 323, 377]"
4,50480003.0,313,32627,8,"[313, 944, 548, 599, 630, 357, 043, 676]"
5,50480004.0,285,97199,10,"[285, 231, 143, 855, 864, 688, 159, 151, 758, 942]"
6,50480005.0,907,28526,25,"[282, 493, 796, 441, 347, 738, 907, 601, 402, 430, 841, 702, 815, 400, 168, 904, 229, 813, 114, 468, 849, 811, 648, 867, 330]"
7,50480006.0,833,30794,14,"[675, 222, 608, 685, 833, 273, 584, 094, 464, 232, 847, 147, 074, 699]"
8,50480007.0,554,29678,18,"[107, 275, 902, 046, 173, 554, 793, 390, 843, 216, 023, 943, 271, 105, 589, 327, 057, 625]"
9,50479002.0,060,31628,11,"[669, 915, 460, 945, 579, 557, 060, 515, 507, 376, 328]"


In [9]:
# Convertimos los valores de la columna 'Numero_Lote' en números enteros
df_sorteo_terrenos['Numero_Lote'] = df_sorteo_terrenos['Numero_Lote'].astype('int64')
# Convertimos los valores de la columna 'boleto_ganador' en cadenas de caracteres
df_sorteo_terrenos['boleto_ganador'] = [f"\'{b}\'" for b in df_sorteo_terrenos['boleto_ganador']]
# Eliminamos las columnas que ya no nos interesan
df_sorteo_terrenos = df_sorteo_terrenos.drop(columns=['antiguedad', 'boletos'])
df_sorteo_terrenos

,Numero_Lote,boleto_ganador,num_empleado
0,50482004,'073',31136
1,50481003,'711',32650
2,50481005,'721',27139
3,50480002,'887',29082
4,50480003,'313',32627
5,50480004,'285',97199
6,50480005,'907',28526
7,50480006,'833',30794
8,50480007,'554',29678
9,50479002,'060',31628
